In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [2]:
!pip install facenet-pytorch

In [28]:
import os
import math
import time
import torch
import random
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from scipy.spatial.distance import cdist
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from facenet_pytorch import InceptionResnetV1
from torch.utils.data import Dataset, random_split
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [4]:
%cd '/content/drive/MyDrive/Colab Notebooks/NN-hw4'

/content/drive/MyDrive/Colab Notebooks/NN-hw4


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1dOPAjSzKRa-cuViwj2xI0QPCDRIAHUON' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1dOPAjSzKRa-cuViwj2xI0QPCDRIAHUON" -O faces && rm -rf /tmp/cookies.txt

^C


In [ ]:
!unzip "./faces"

Archive:  ./faces
replace Faces/Alexandra Daddario/Alexandra Daddario_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [29]:
random.seed(111)

# Face Classifier

## Data Loader

In [6]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

In [7]:
data_path = '/content/drive/MyDrive/Colab Notebooks/NN-hw4/Faces'

In [8]:
dataset = ImageFolder(root=data_path, transform=transform)

In [9]:
# Split the dataset into training and testing sets
total_size = len(dataset)
train_size = int(0.8 * total_size)  # 80% train
test_size = total_size - train_size  # 20% test

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [10]:
# Create a DataLoader for batch processing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
len(dataset.classes)

14

## MLP

In [12]:
class MLP1(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP1, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:
# drop out and batch normalization
class MLP2(nn.Module):
    def __init__(self, input_size, hidden_size=256, output_size=14, dropout_prob=0.5):
        super(MLP2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [14]:
# larger model
class MLP3(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [15]:
# larger model + dropout
class MLP4(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.drop_out1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.drop_out2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.drop_out1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.drop_out2(x)
        x = self.fc3(x)
        return x

In [16]:
# larger model + bn
class MLP5(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.batch_norm1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batch_norm2(x)
        x = self.fc3(x)
        return x

In [17]:
# larger model + dropout and bn
class MLP6(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.drop_out1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.drop_out2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.batch_norm1(x)
        x = self.drop_out1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batch_norm2(x)
        x = self.drop_out2(x)
        x = self.fc3(x)
        return x

## Model

In [40]:
# Set the input size and output size of the MLP
input_size = 512
output_size = 14

In [41]:
# mlp = MLP1(input_size=input_size, output_size=output_size)
# mlp = MLP2(input_size=input_size, output_size=output_size)
# mlp = MLP3(input_size=input_size, output_size=output_size)
# mlp = MLP4(input_size=input_size, output_size=output_size)
# mlp = MLP5(input_size=input_size, output_size=output_size)
mlp = MLP6(input_size=input_size, output_size=output_size)

In [42]:
inception_resnet = InceptionResnetV1(pretrained='vggface2', classify=False)
inception_resnet.to(device)
inception_resnet.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [22]:
# Fine Tune
# Freeze all parameters except the last layer nad logits
for name, param in inception_resnet.named_parameters():
    if ('last' not in name) and ('logits' not in name):
        param.requires_grad = False

In [34]:
# Freeze inception_resnet
for _, param in inception_resnet.named_parameters():
    param.requires_grad = False

In [43]:
combined_model = nn.Sequential(inception_resnet, mlp).to(device)

In [44]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(combined_model.parameters(), lr=0.001, momentum=0.9)

In [45]:
tr_num_batches = len(train_loader)
tr_num_samples = len(train_loader.dataset)
te_num_batches = len(test_loader)
te_num_samples = len(test_loader.dataset)
acc_history = {'train': [], 'test': []}
loss_history = {'train': [], 'test': []}
best_loss = float('inf')
best_accuracy = 0
counter = 0

In [46]:
# Train
num_epochs = 5

start_time = time.time()
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    combined_model.train()
    correct = 0
    total_loss = 0.0
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = combined_model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        _, predictions = torch.max(outputs, dim=1)
        correct += (predictions == labels).sum().item()
        total_loss += loss.item()
        running_loss += loss.item()
        # if (i+1) % 10 == 0:
          # print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/10:.4f}')
        running_loss = 0.0


    epoch_accuracy = (correct / tr_num_samples) * 100
    epoch_loss = total_loss / tr_num_batches
    acc_history['train'].append(epoch_accuracy)
    loss_history['train'].append(epoch_loss)
    print(f'Train - Accuracy: {epoch_accuracy}, Loss: {epoch_loss}')

print('Training finished!')
end_time = time.time()
total_time = (end_time - start_time)
print(f'training time = {total_time} seconds')

Epoch 1/5
Train - Accuracy: 60.689655172413794, Loss: 1.5090776230837848
Epoch 2/5
Train - Accuracy: 98.36206896551725, Loss: 0.27780865636226293
Epoch 3/5
Train - Accuracy: 99.6551724137931, Loss: 0.13883619735369812
Epoch 4/5
Train - Accuracy: 99.74137931034483, Loss: 0.09622016871297681
Epoch 5/5
Train - Accuracy: 99.82758620689656, Loss: 0.0678040311747306
Training finished!
training time = 1428.956758737564 seconds


In [48]:
# Test
combined_model.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = combined_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {(100 * correct / total):.2f}%')

Test Accuracy: 100.00%


# Face Verifier

## Split Test Dataset

In [49]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

In [50]:
data_path = '/content/drive/MyDrive/Colab Notebooks/NN-hw4/Faces'

In [51]:
dataset = ImageFolder(root=data_path, transform=transform)

In [52]:
total_size = len(dataset)
test1_size = int(0.1 * total_size)
test2_size = int(0.1 * total_size)
train_size = total_size - test1_size - test2_size

test1_dataset, test2_dataset, train_dataset = random_split(dataset, [test1_size, test2_size, train_size])

In [53]:
batch_size = 32
known_loader = DataLoader(test1_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
unknown_loader = DataLoader(test2_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

## Calculate embeddings

In [54]:
inception_resnet.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [55]:
known_labels = []
known_embeddings = []
with torch.no_grad():
    for inputs, labels in known_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        embeddings = inception_resnet(inputs)
        known_labels.append(labels)
        known_embeddings.append(embeddings)

In [56]:
unknown_labels = []
unknown_embeddings = []
with torch.no_grad():
    for inputs, labels in unknown_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        embeddings = inception_resnet(inputs)
        unknown_labels.append(labels)
        unknown_embeddings.append(embeddings)

In [57]:
known_em = torch.cat((known_embeddings[0], known_embeddings[1], known_embeddings[2], known_embeddings[3]), dim=0)
unknown_em = torch.cat((unknown_embeddings[0], unknown_embeddings[1], unknown_embeddings[2], unknown_embeddings[3]), dim=0)
known_label = torch.cat((known_labels[0], known_labels[1], known_labels[2], known_labels[3]), dim=0)
unknown_label = torch.cat((unknown_labels[0], unknown_labels[1], unknown_labels[2], unknown_labels[3]), dim=0)

In [60]:
known_em = known_embeddings[0]
unknown_em = unknown_embeddings[0]
known_label = known_labels[0]
unknown_label = unknown_labels[0]

## Calculate similarity

In [61]:
closest_label = []
for i in range(len(known_em)):
  unknown_em = unknown_em.cpu()
  known_em = known_em.cpu()
  em = known_em[i]

  # cosine similarity
  similarities = cosine_similarity([em], unknown_em)
  closest_index = np.argmax(similarities)

  # euclidean distance
  distance = cdist ([em], unknown_em, 'euclidean')
  # closest_index = np.argmin(distance)

  if similarities.all() < 0 :
    print("no close matches found !")

  label = unknown_label[closest_index]
  closest_label.append(label)

closest_label = torch.stack(closest_label,dim=0)

In [145]:
for i in range(len(known_label)):
  print(f'known label: {known_label[i]}, closest label: {closest_label[i]}')

known label: 13, closest label: 13
known label: 11, closest label: 11
known label: 4, closest label: 4
known label: 13, closest label: 13
known label: 2, closest label: 2
known label: 12, closest label: 12
known label: 8, closest label: 8
known label: 3, closest label: 3
known label: 9, closest label: 9
known label: 1, closest label: 1
known label: 12, closest label: 12
known label: 10, closest label: 10
known label: 7, closest label: 7
known label: 12, closest label: 12
known label: 3, closest label: 3
known label: 12, closest label: 12
known label: 8, closest label: 8
known label: 2, closest label: 2
known label: 6, closest label: 6
known label: 7, closest label: 7
known label: 9, closest label: 9
known label: 2, closest label: 2
known label: 0, closest label: 0
known label: 3, closest label: 3
known label: 5, closest label: 5
known label: 5, closest label: 5
known label: 13, closest label: 13
known label: 2, closest label: 2
known label: 4, closest label: 4
known label: 10, closest 

In [62]:
equals = (known_label == closest_label).sum()
correct = equals / len(known_label) * 100
print(f'{correct}% of labels are correct')

96.875% of labels are correct
